In [21]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from fancyimpute import IterativeImputer
from sklearn.metrics import mean_absolute_error, mean_squared_error

import numpy as np
import pandas as pd

import pdb
import pickle
import random

def load_data(data_path='./data/20221122_raw_data_utf-8.csv', classification = True, args=None):
    # load data
    df = pd.read_csv(data_path, index_col=0)       # [8844, 37]

    # Making y label
    cdc_count = df['index_id_cdc'].value_counts()
    cdc_count = cdc_count.to_frame().reset_index()
    cdc_count.columns = ['id_inch', 'index_id_cdc_num']
    if classification:
        over = cdc_count['index_id_cdc_num']>=6
        cdc_count['index_id_cdc_num'][over] = 6

    df = pd.merge(df, cdc_count, how='left', on='id_inch')      # left outer join
    df['index_id_cdc_num'] = df['index_id_cdc_num'].fillna(0)   # 결측값 0으로 대치

    replace_missing_value='mice'
    if replace_missing_value == 'replace':
        for i in range(1, 16):
            col_name = f'a{i}'
            df.loc[:, [col_name]] = df.loc[:, [col_name]].fillna(2)
    elif replace_missing_value == 'binomial':
        pass
    elif replace_missing_value == 'mice':
        def custom_transform(value):
            if value < 0:
                return 0
            elif value <= 0.5:
                return 0
            else:
                return 1
        imputer = IterativeImputer(random_state=42)
        selected_columns = [f'a{i}' for i in range(1, 16)]
        selected_df = df[selected_columns]
        completed_selected_df = imputer.fit_transform(selected_df)
        df[selected_columns] = completed_selected_df
        for col in selected_columns:
            df[col] = df[col].apply(custom_transform)
            
    else:   
        df.dropna(how='any', subset=['contact_n'], inplace=True)    # 차수가 없는것 삭제
        df = df.drop(df[df.contact_n == '?'].index)

        df.dropna(how='any', subset=['sex'], inplace=True)          # 성별 없는것 삭제

        df.dropna(how='any', subset=['a1','a2','a3','a4','a5','a6','a7','a8','a9','a10','a11','a12','a13','a14','a15'], inplace=True)   # 증상 (a1-15) 없는것 삭제

    # index reset
    df = df.reset_index(drop=True)                        # [1154, 38]
    unique_group = pd.unique(df['transmission_route'])    # 155
    return df, unique_group



In [22]:
df,_ = load_data(data_path='./20221122_raw_data_utf-8.csv')

/tmp/ipykernel_2617031/3069765885.py:25: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  cdc_count['index_id_cdc_num'][over] = 6
/tmp/ipykernel_2617031/3069765885.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice

In [23]:
df.head()

,id_inch,sex,birthyear,symptom_date,confirm_date,test_date_last,test_R,test_E,test_N,test_O,...,a7,a8,a9,a10,a11,a12,a13,a14,a15,index_id_cdc_num
0,인천2,여,1959,NaN,2020-02-22,2020-02-21,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,1,0.0
1,인천4,여,1991,NaN,2020-02-28,2020-02-27,33.5151,30.8225,NaN,NaN,...,0,0,0,0,1,0,0,0,0,0.0
2,인천5,남,1956,NaN,2020-02-28,2020-02-27,22.0100,18.82,NaN,NaN,...,1,1,0,1,0,0,0,0,0,1.0
3,인천7,남,1963,NaN,2020-03-01,2020-03-01,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,1,0.0
4,인천8,여,1972,NaN,2020-03-03,2020-03-02,NaN,NaN,NaN,NaN,...,1,0,0,0,0,0,0,0,0,0.0
